<h1><center>Random Forest Models</center></h1>

<hr>

# Bag-of-Words Vectoriser
## Standard Metrics

In [1]:
import spacy
import pandas as pd
import re
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import string
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import nltk
from sklearn.pipeline import Pipeline

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Reading .csv file

df_train = pd.read_csv(r"C:\Users\nathi_000\Desktop\Python Files\NLP Project\nlp-web-scrapping\data\raw\tweets-train.csv")

df_test = pd.read_csv(r"C:\Users\nathi_000\Desktop\Python Files\NLP Project\nlp-web-scrapping\data\raw\tweets-test.csv")

X_train = df_train['text'].astype(str)
Y_train = df_train['sentiment'].astype(str)

X_test = df_test['text'].astype(str)
Y_test = df_test['sentiment'].astype(str)

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Creating custom tokeniser and cleaning function

Lemmatiser = nltk.stem.WordNetLemmatizer()
# Instantiating the NLTK Lemmatiser

punctuations = string.punctuation
# Putting punctuation symbols into an object

nlp = spacy.load("en_core_web_sm")
# Import spacy model

stopwords = spacy.lang.en.stop_words.STOP_WORDS
# A list of stopwords that can be filtered out
    # NLTK also has a stop words object but it has fewer words

def text_cleaner(sentence):    
                
    sentence = "".join([char for char in sentence.strip() if char not in punctuations])
    # Getting rid of any punctuation characters
    
    myTokens = re.split('\W+', sentence)
    # Tokenising the words
    
    myTokens = [token.lower() for token in myTokens if token not in stopwords]
    # Removing stop words
    
    myTokens = [Lemmatiser.lemmatize(token) for token in myTokens]
    # Lemmatising the words and putting in lower case except for proper nouns
    
    return myTokens    

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Creating Vectoriser and Classifiers

bow_vector = CountVectorizer(tokenizer = text_cleaner, ngram_range=(1,1))

rfc = RandomForestClassifier(n_jobs=-1)

pipe = Pipeline([('vectorizer', bow_vector)
                 ,('classifier', rfc)])

pipe.fit(X_train, Y_train)

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Evaluating the model

predicted = pipe.predict(X_test)

# Evaluation
print("RAW DATA - BoW, Metrics:\n")
print("Logistic Regression Accuracy:\n",metrics.accuracy_score(Y_test, predicted),"\n") # Accuracy
print("Logistic Regression Precision:\n",metrics.precision_score(Y_test, predicted, average='macro'),"\n") # Precision
print("Logistic Regression Recall:\n",metrics.recall_score(Y_test, predicted, average='macro'),"\n") # Recall
print("Logistic Regression F1 Score:\n",metrics.f1_score(Y_test, predicted, average='macro')) # F1 Score

RAW DATA - BoW, Metrics:

Logistic Regression Accuracy:
 0.700339558573854 

Logistic Regression Precision:
 0.707890521479854 

Logistic Regression Recall:
 0.6974185379443766 

Logistic Regression F1 Score:
 0.7014634744052103


<hr>

# Bag-of-Words Vectoriser
## K-Fold Evaluation

In [4]:
import spacy
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import string
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import nltk
from sklearn.pipeline import Pipeline

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Reading .csv file

df = pd.read_csv(r"C:\Users\nathi_000\Desktop\Python Files\NLP Project\nlp-web-scrapping\data\raw\tweets-train.csv")

X = df['text'].astype(str)
# Convert to type 'string' as pandas converts inputs to their most relevant type
    # The issue is sometimes pandas converts data to a 'float'; this doesn't work with the evaluation functions

Y = df['sentiment'].astype(str)

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Creating custom tokeniser and cleaning function

Lemmatiser = nltk.stem.WordNetLemmatizer()
# Instantiating the NLTK Lemmatiser

punctuations = string.punctuation
# Putting punctuation symbols into an object

nlp = spacy.load("en_core_web_sm")
# Import spacy model

stopwords = spacy.lang.en.stop_words.STOP_WORDS
# A list of stopwords that can be filtered out
    # NLTK also has a stop words object but it has fewer words

def text_cleaner(sentence):    
                
    sentence = "".join([char for char in sentence.strip() if char not in punctuations])
    # Getting rid of any punctuation characters
    
    myTokens = re.split('\W+', sentence)
    # Tokenising the words
    
    myTokens = [token.lower() for token in myTokens if token not in stopwords]
    # Removing stop words
    
    myTokens = [Lemmatiser.lemmatize(token) for token in myTokens]
    # Lemmatising the words and putting in lower case except for proper nouns
    
    return myTokens    

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Creating Vectoriser and Classifiers

bow_vector = CountVectorizer(tokenizer = text_cleaner, ngram_range=(1,1))

rfc = RandomForestClassifier(n_jobs=-1)

pipe = Pipeline([('vectorizer', bow_vector)
                 ,('classifier', rfc)])

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Evaluating the model

k_fold = KFold(n_splits=5)
# 5 splits

print("RAW DATA - BoW, K-Fold:\n")
print(cross_val_score(estimator=pipe, X=X, y=Y, cv=k_fold, scoring='accuracy', n_jobs=-1))

RAW DATA - BoW, K-Fold:

[0.69092232 0.69796215 0.694869   0.69432314 0.70178311]


<hr>

# TF-IDF Vectoriser
## Standard Metrics

In [2]:
import spacy
import pandas as pd
import re
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import string
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import nltk
from sklearn.pipeline import Pipeline

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Reading .csv file

df_train = pd.read_csv(r"C:\Users\nathi_000\Desktop\Python Files\NLP Project\nlp-web-scrapping\data\raw\tweets-train.csv")

df_test = pd.read_csv(r"C:\Users\nathi_000\Desktop\Python Files\NLP Project\nlp-web-scrapping\data\raw\tweets-test.csv")

X_train = df_train['text'].astype(str)
Y_train = df_train['sentiment'].astype(str)

X_test = df_test['text'].astype(str)
Y_test = df_test['sentiment'].astype(str)

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Creating custom tokeniser and cleaning function

Lemmatiser = nltk.stem.WordNetLemmatizer()
# Instantiating the NLTK Lemmatiser

punctuations = string.punctuation
# Putting punctuation symbols into an object

nlp = spacy.load("en_core_web_sm")
# Import spacy model

stopwords = spacy.lang.en.stop_words.STOP_WORDS
# A list of stopwords that can be filtered out
    # NLTK also has a stop words object but it has fewer words

def text_cleaner(sentence):    
                
    sentence = "".join([char for char in sentence.strip() if char not in punctuations])
    # Getting rid of any punctuation characters
    
    myTokens = re.split('\W+', sentence)
    # Tokenising the words
    
    myTokens = [token.lower() for token in myTokens if token not in stopwords]
    # Removing stop words
    
    myTokens = [Lemmatiser.lemmatize(token) for token in myTokens]
    # Lemmatising the words and putting in lower case except for proper nouns
    
    return myTokens    

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Creating Vectoriser and Classifiers

tfidf_vector = TfidfVectorizer(tokenizer = text_cleaner)

rfc = RandomForestClassifier(n_jobs=-1)

pipe = Pipeline([('vectorizer', tfidf_vector)
                 ,('classifier', rfc)])

pipe.fit(X_train, Y_train)

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Evaluating the model

predicted = pipe.predict(X_test)

# Evaluation
print("RAW DATA - TF-IDF, Metrics:\n")
print("Logistic Regression Accuracy:\n",metrics.accuracy_score(Y_test, predicted),"\n") # Accuracy
print("Logistic Regression Precision:\n",metrics.precision_score(Y_test, predicted, average='macro'),"\n") # Precision
print("Logistic Regression Recall:\n",metrics.recall_score(Y_test, predicted, average='macro'),"\n") # Recall
print("Logistic Regression F1 Score:\n",metrics.f1_score(Y_test, predicted, average='macro')) # F1 Score

RAW DATA - TF-IDF, Metrics:

Logistic Regression Accuracy:
 0.7051499717034522 

Logistic Regression Precision:
 0.714771804687231 

Logistic Regression Recall:
 0.7010965764365583 

Logistic Regression F1 Score:
 0.706177608947038


<hr>

# TF-IDF Vectoriser
## K-Fold Evaluation

In [3]:
import spacy
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import string
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
import nltk
from sklearn.pipeline import Pipeline

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Reading .csv file

df = pd.read_csv(r"C:\Users\nathi_000\Desktop\Python Files\NLP Project\nlp-web-scrapping\data\raw\tweets-train.csv")

X = df['text'].astype(str)
# Convert to type 'string' as pandas converts inputs to their most relevant type
    # The issue is sometimes pandas converts data to a 'float'; this doesn't work with the evaluation functions

Y = df['sentiment'].astype(str)

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Creating custom tokeniser and cleaning function

Lemmatiser = nltk.stem.WordNetLemmatizer()
# Instantiating the NLTK Lemmatiser

punctuations = string.punctuation
# Putting punctuation symbols into an object

nlp = spacy.load("en_core_web_sm")
# Import spacy model

stopwords = spacy.lang.en.stop_words.STOP_WORDS
# A list of stopwords that can be filtered out
    # NLTK also has a stop words object but it has fewer words

def text_cleaner(sentence):    
                
    sentence = "".join([char for char in sentence.strip() if char not in punctuations])
    # Getting rid of any punctuation characters
    
    myTokens = re.split('\W+', sentence)
    # Tokenising the words
    
    myTokens = [token.lower() for token in myTokens if token not in stopwords]
    # Removing stop words
    
    myTokens = [Lemmatiser.lemmatize(token) for token in myTokens]
    # Lemmatising the words and putting in lower case except for proper nouns
    
    return myTokens    

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Creating Vectoriser

tfidf_vector = TfidfVectorizer(tokenizer = text_cleaner)

#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-

                        # Evaluating the model
    
rfc = RandomForestClassifier(n_jobs=-1)

pipe = Pipeline([('vectorizer', tfidf_vector)
                 ,('classifier', rfc)])

k_fold = KFold(n_splits=5)
# 5 splits

print("RAW DATA - TF-IDF, K-Fold:\n")
print(cross_val_score(estimator=pipe, X=X, y=Y, cv=k_fold, scoring='accuracy', n_jobs=-1))

RAW DATA - TF-IDF, K-Fold:

[0.69474259 0.70342067 0.70742358 0.70160116 0.70178311]
